In [1]:
from transformers import AutoTokenizer, AutoModel, logging
from collections import defaultdict
from itertools import combinations
from tqdm.notebook import tqdm
import pandas as pd
import torch
import os

logging.set_verbosity(50)

In [2]:
model_checkpoint = 'roberta-base'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.vocab['party']

In [5]:
df = pd.read_csv(os.path.abspath('../../data/sense_data/sense_data-cuad.csv'))

In [7]:
df.dropna(inplace=True)

In [ ]:
#With this code, we can verify whether each example has only 1 occurrence of the target word.
#This helps to ensure that we don't have multiple instances of contextualization in the same sentence.
for row in df.itertuples():
    word = row.word.lower()
    sentence = row.example
    tokenized_idx = tokenizer(sentence)['input_ids']
    if tokenized_idx.count(tokenizer.vocab[word]) != 1:
        print(row)
        break

In [ ]:
#Using pooler output

cos = torch.nn.CosineSimilarity()
sim_scores = defaultdict(list)

for word in tqdm(df['word'].unique()):
    word_indices = df[df['word'] == word].index
    for comb in list(combinations(list(range(word_indices[0], word_indices[-1]+1)), 2)):
        indexA = comb[0]
        indexB = comb[1]

        tokenized_inputA = tokenizer(df.iloc[indexA].example, return_tensors='pt') 
        pooler_outputA = model(**tokenized_inputA).pooler_output

        tokenized_inputB = tokenizer(df.iloc[indexB].example, return_tensors='pt') 
        pooler_outputB = model(**tokenized_inputB).pooler_output

        sim_scores[(word, df.iloc[indexA].sense_def, df.iloc[indexB].sense_def)].append(\
            cos(pooler_outputA, pooler_outputB).item())
    break

In [16]:
#Using contextualized entity output (BERTs/RoBERTa)

cos = torch.nn.CosineSimilarity(dim=0)
sim_scores = defaultdict(list)

def find_vocab_idx(word, tokenization):
    if 'roberta' in model_checkpoint:
        if word in tokenizer.vocab.keys():
            if tokenizer.vocab[word] in tokenization['input_ids'].tolist()[0]:
                return tokenizer.vocab[word]
            elif ('Ġ'+ word.lower()) in tokenizer.vocab.keys():
                if tokenizer.vocab['Ġ'+ word.lower()] in tokenization['input_ids'].tolist()[0]:
                    word = 'Ġ' + word.lower()
                    return tokenizer.vocab[word]
                else:
                    word = 'Ġ' + word
                    return tokenizer.vocab[word]
    else:
        if word in tokenizer.vocab.keys():
            if tokenizer.vocab[word] in tokenization['input_ids'].tolist()[0]:
                return tokenizer.vocab[word]
            else:
                return tokenizer.vocab[word.lower()]

for word in tqdm(df['word'].unique()):
    word_indices = df[df['word'] == word].index
    for comb in tqdm(list(combinations(list(range(word_indices[0], word_indices[-1]+1)), 2))):
        indexA = comb[0]
        indexB = comb[1]        
        
        tokenized_inputA = tokenizer(df.iloc[indexA].example, return_tensors='pt') 
        contextualized_embeddingsA = model(**tokenized_inputA).last_hidden_state

        tokenized_inputB = tokenizer(df.iloc[indexB].example, return_tensors='pt') 
        contextualized_embeddingsB = model(**tokenized_inputB).last_hidden_state

        wordA_vocab_idx = find_vocab_idx(df.iloc[indexA].word, tokenized_inputA)
        wordB_vocab_idx = find_vocab_idx(df.iloc[indexB].word, tokenized_inputB)
        
        entity_embeddingA = contextualized_embeddingsA[0][tokenized_inputA['input_ids'].tolist()[0].index(wordA_vocab_idx)]       
        entity_embeddingB = contextualized_embeddingsB[0][tokenized_inputB['input_ids'].tolist()[0].index(wordB_vocab_idx)]
        
        sim_scores[(word, df.iloc[indexA].sense_def, df.iloc[indexB].sense_def)].append(\
            cos(entity_embeddingA, entity_embeddingB).item())

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

Ġcompany 1260 [0, 42390, 24008, 368, 5658, 45, 696, 6, 5780, 50, 27369, 877, 143, 335, 59, 5, 9467, 11, 5, 78, 7702, 2041, 396, 5, 5486, 1982, 7132, 9, 5, 1260, 4, 2]
Ġcompany 1260 [0, 133, 26774, 368, 5658, 45, 9263, 42, 1288, 4682, 2115, 7132, 9, 1260, 4, 2]


ValueError: None is not in list

In [ ]:
#Using contextualized entity ouptut (SenseBERT)
import tensorflow as tf
from sensebert import SenseBert

with tf.Session() as session:
    sensebert_model = SenseBert("../sensebert-base-uncased", session=session)  # or sensebert-large-uncased
    input_ids, input_mask = sensebert_model.tokenize(["I went to the store to buy some groceries.", "The store was closed."])
    model_outputs = sensebert_model.run(input_ids, input_mask)

contextualized_embeddings, mlm_logits, supersense_logits = model_outputs  # these are NumPy arrays

In [ ]:
import numpy as np
for key, val in sim_scores.items():
    print(key, np.round(torch.mean(torch.Tensor(val)).item(), 2))